### Compute the input embeddings and writes to output files


In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

from utils.utils import *
from utils.tt_helpers import *
from sentence_transformers import SentenceTransformer
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # Suppress huggingface warning

sbert = SentenceTransformer('all-mpnet-base-v2') #  109 M param

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/Users/ozkansafak/code/retrieval/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
# # load ids_all from disk
# with open('output/ids_all.pkl', 'rb') as file:
#     ids_all = pickle.load(file)

# load ids_toys from disk
with open('output/ids_toys.pkl', 'rb') as file:
    ids_toys = pickle.load(file)

# load queries and product_info from disk
queries, _, _ = load_raw_queries()
with open('output/product_info.json', 'r') as file:
    product_info = json.load(file)


loaded (queries, query_embeddings, product_embeddings) from  output/raw_queries.pkl


In [18]:
len(query_embeddings)

19860

In [19]:
# Compute embeddings via inference on SBERT
# INPUTS:
load = True
istart = 0
# ------------------------------


start = time.time()
if load:
    # load all queries embedded thus far
    queries, query_embeddings, product_embeddings = load_raw_queries()
else:
    product_embeddings = dict()
    query_embeddings = dict()
    
istart = len(query_embeddings)
print(f'istart: {istart}')

for i, _id in enumerate(ids_toys[istart:], start=istart):
    name, details, description = product_info[_id]['name'], product_info[_id]['details'], product_info[_id]['description']
    sentence = f"""Product Name:"{name}", Product Details:"{details}", Product Description:"{description}" """
    
    product_embeddings[_id] = sbert.encode(sentence)
    query_embeddings[_id] = sbert.encode(queries[_id])

    print('.', end='')
    if (i+1) % 100 == 0:
        print(f" i:{i} {print_runtime(start, False)}")
        write_raw_queries(queries, query_embeddings, product_embeddings)


loaded (queries, query_embeddings, product_embeddings) from  output/raw_queries.pkl
istart: 19800
.................................................................................................... i:19899 Runtime: 0 min 2 sec
written to output/raw_queries.pkl: Runtime: 0 min 0 sec
.................................................................................................... i:19999 Runtime: 0 min 5 sec
written to output/raw_queries.pkl: Runtime: 0 min 0 sec
.................................................................................................... i:20099 Runtime: 0 min 8 sec
written to output/raw_queries.pkl: Runtime: 0 min 0 sec
.................................................................................................... i:20199 Runtime: 0 min 12 sec
written to output/raw_queries.pkl: Runtime: 0 min 0 sec
.................................................................................................... i:20299 Runtime: 0 min 14 sec
written to output/raw_quer

KeyError: '65862524c515d2f5de760fdd'

### query `name`, `details` and `description` and store in a local json file

In [ ]:
# run this in a cell of its own
info = dict()

In [164]:
start = time.time()

istart = len(info)
print(f'istart: {istart}')

for i, _id in enumerate(ids_toys[istart:], start=istart):
    product_info = query_by_id(_id)
    name, details, description = get_product_info_from_dict(product_info)
    
    info[_id] = dict()
    info[_id]['name'] = name
    info[_id]['details'] = details
    info[_id]['description'] = description
    print('.', end='')
    if i % 1000 == 0:
        print(i)
        print_runtime(start)
        
with open('output/product_info.json', 'w') as f:
    json.dump(info, f)

print_runtime(start)


istart: 23865
........................................................................................................................................24000
Runtime: 0 min 8 sec
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [165]:
with open('output/product_info.json', 'w') as f:
    json.dump(info, f)


In [ ]:
# stack up a list of tensors into single query (qb) and product (xb) tensors.
def construct_qb_xb(query_embeddings, product_embeddings, ids_toys):
    qb = torch.stack([torch.tensor(query_embeddings[_id]) for _id in ids_toys])
    xb = torch.tensor(np.array([product_embeddings[_id] for _id in ids_toys]))
    
    # Save tensors and lists and dicts
    torch.save({'qb': qb, 'xb': xb}, 'output/qb_xb.pt')
    write_raw_queries(queries, query_embeddings, product_embeddings, pids)
    
    # Double check if the shapes look correct.
    print(qb.shape, xb.shape, qb.requires_grad, xb.requires_grad)
    print(qb, xb)

qb, xb = construct_qb_xb(query_embeddings, product_embeddings, ids_toys)
